# creating first Desktop application for Employee management system using PyQt5

### require packages are

1. PyQt5 - pip install PyQt5

2. MySQLConnector and MySqlClient

In [1]:
import sys
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QLabel, QLineEdit, QComboBox, 
    QPushButton, QMessageBox, QTableWidget, QTableWidgetItem, QTextEdit,
    QStackedWidget, QAction, QMenu, QMenuBar, QToolBar, QStatusBar,
    QHBoxLayout, QVBoxLayout, QAbstractItemView, QHeaderView, QFormLayout,
    QGroupBox, QDialog, QDialogButtonBox, QFrame, QSizePolicy
)
from PyQt5.QtGui import QIcon, QFont, QPalette, QColor, QPixmap
from PyQt5.QtCore import Qt, QSize

class WelcomePage(QWidget):
    def __init__(self):
        super().__init__()
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        # Welcome message
        welcome_label = QLabel("Welcome to ABC Employee Management System")
        welcome_label.setFont(QFont("Arial", 24, QFont.Bold))
        welcome_label.setAlignment(Qt.AlignCenter)
        welcome_label.setStyleSheet("color: #2c3e50; margin: 30px 0;")
        
        # Company description
        description = QLabel(
            "ABC Corp. is a leading technology company specializing in software solutions.\n"
            "Our Employee Management System helps you efficiently manage your workforce."
        )
        description.setFont(QFont("Arial", 14))
        description.setAlignment(Qt.AlignCenter)
        description.setStyleSheet("color: #7f8c8d; margin-bottom: 40px;")
        
        # Features section
        features_box = QGroupBox("System Features")
        features_box.setFont(QFont("Arial", 14, QFont.Bold))
        features_box.setStyleSheet("QGroupBox { border: 2px solid #3498db; border-radius: 10px; margin: 20px; }")
        
        features_layout = QVBoxLayout()
        
        features = [
            "➤ Add new employees with complete details",
            "➤ Update existing employee information",
            "➤ Delete employee records as needed",
            "➤ View individual employee profiles",
            "➤ Browse the complete employee list",
            "➤ Search and filter employee data"
        ]
        
        for feature in features:
            label = QLabel(feature)
            label.setFont(QFont("Arial", 12))
            features_layout.addWidget(label)
        
        features_box.setLayout(features_layout)
        
        # Add widgets to main layout
        layout.addWidget(welcome_label)
        layout.addWidget(description)
        layout.addStretch(1)
        layout.addWidget(features_box)
        layout.addStretch(1)


class AddEmployeeWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        main_layout = QVBoxLayout()
        self.setLayout(main_layout)
        
        # Title
        title = QLabel("Add New Employee")
        title.setFont(QFont("Arial", 18, QFont.Bold))
        title.setStyleSheet("color: #2c3e50; margin: 10px 0 20px 0;")
        title.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(title)
        
        # Form layout
        form_group = QGroupBox("Employee Information")
        form_group.setFont(QFont("Arial", 12))
        form_layout = QFormLayout()
        form_layout.setSpacing(15)
        form_layout.setContentsMargins(20, 20, 20, 20)
        form_group.setLayout(form_layout)
        
        # Form fields
        label_font = QFont("Arial", 11)
        
        # Name
        self.name = QLineEdit()
        self.name.setPlaceholderText("Enter full name")
        self.name.setFont(label_font)
        self.name.setStyleSheet("padding: 8px;")
        
        # Email
        self.email = QLineEdit()
        self.email.setPlaceholderText("example@company.com")
        self.email.setFont(label_font)
        self.email.setStyleSheet("padding: 8px;")
        
        # Phone
        self.phone = QLineEdit()
        self.phone.setPlaceholderText("123-456-7890")
        self.phone.setFont(label_font)
        self.phone.setStyleSheet("padding: 8px;")
        
        # Designation
        self.designation = QComboBox()
        self.designation.addItems(["Team Leader", "HR", "Developer", "Tester", "Designer", "Manager", "Other"])
        self.designation.setFont(label_font)
        self.designation.setStyleSheet("padding: 8px;")
        
        # Add fields to form
        form_layout.addRow(QLabel("Full Name:"), self.name)
        form_layout.addRow(QLabel("Email:"), self.email)
        form_layout.addRow(QLabel("Phone:"), self.phone)
        form_layout.addRow(QLabel("Designation:"), self.designation)
        
        # Submit button
        self.submitBtn = QPushButton("Add Employee")
        self.submitBtn.setFont(QFont("Arial", 12, QFont.Bold))
        self.submitBtn.setStyleSheet(
            "background-color: #27ae60; color: white; padding: 10px 20px; border-radius: 5px;"
        )
        self.submitBtn.clicked.connect(self.addEmployeeDB)
        
        # Add widgets to main layout
        main_layout.addWidget(form_group)
        main_layout.addWidget(self.submitBtn, alignment=Qt.AlignCenter)
        main_layout.addStretch(1)
        
    def addEmployeeDB(self):
        # Validate required fields
        if not all([self.name.text().strip(), 
                    self.email.text().strip(), 
                    self.phone.text().strip()]):
            QMessageBox.warning(self, "Error", "Please fill all required fields!")
            return

        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Parameterized query to prevent SQL injection
            query = """
                INSERT INTO emp 
                (name, email, phone, Designation) 
                VALUES (%s, %s, %s, %s)
            """
            values = (
                self.name.text(),
                self.email.text(),
                self.phone.text(),
                self.designation.currentText()
            )
            
            cur.execute(query, values)
            conn.commit()  # Save changes to database
            
            QMessageBox.information(
                self, 
                "Success", 
                "Employee added successfully!"
            )
            
            # Clear form after successful submission
            self.name.clear()
            self.email.clear()
            self.phone.clear()
            self.designation.setCurrentIndex(0)
            
        except mysql.connector.Error as err:
            QMessageBox.critical(
                self, 
                "Database Error", 
                f"Error: {err}"
            )
        finally:
            # Clean up resources
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()


class UpdateEmployeeWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        main_layout = QVBoxLayout()
        self.setLayout(main_layout)
        
        # Title
        title = QLabel("Update Employee Details")
        title.setFont(QFont("Arial", 18, QFont.Bold))
        title.setStyleSheet("color: #2c3e50; margin: 10px 0 20px 0;")
        title.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(title)
        
        # Search section
        search_group = QGroupBox("Search Employee")
        search_group.setFont(QFont("Arial", 12))
        search_layout = QHBoxLayout()
        search_layout.setContentsMargins(15, 15, 15, 15)
        search_group.setLayout(search_layout)
        
        # Search input
        self.empid = QLineEdit()
        self.empid.setPlaceholderText("Enter Employee ID")
        self.empid.setFont(QFont("Arial", 11))
        self.empid.setStyleSheet("padding: 8px;")
        
        # Search button
        self.searchBtn = QPushButton("Search")
        self.searchBtn.setFont(QFont("Arial", 11))
        self.searchBtn.setStyleSheet(
            "background-color: #3498db; color: white; padding: 8px 20px; border-radius: 5px;"
        )
        self.searchBtn.clicked.connect(self.searchEmployee)
        
        search_layout.addWidget(QLabel("Employee ID:"))
        search_layout.addWidget(self.empid)
        search_layout.addWidget(self.searchBtn)
        
        # Form section
        form_group = QGroupBox("Employee Information")
        form_group.setFont(QFont("Arial", 12))
        form_layout = QFormLayout()
        form_layout.setSpacing(15)
        form_layout.setContentsMargins(20, 20, 20, 20)
        form_group.setLayout(form_layout)
        
        # Form fields
        label_font = QFont("Arial", 11)
        
        # Name
        self.name = QLineEdit()
        self.name.setPlaceholderText("Enter full name")
        self.name.setFont(label_font)
        self.name.setStyleSheet("padding: 8px;")
        
        # Email
        self.email = QLineEdit()
        self.email.setPlaceholderText("example@company.com")
        self.email.setFont(label_font)
        self.email.setStyleSheet("padding: 8px;")
        
        # Phone
        self.phone = QLineEdit()
        self.phone.setPlaceholderText("123-456-7890")
        self.phone.setFont(label_font)
        self.phone.setStyleSheet("padding: 8px;")
        
        # Designation
        self.designation = QComboBox()
        self.designation.addItems(["Team Leader", "HR", "Developer", "Tester", "Designer", "Manager", "Other"])
        self.designation.setFont(label_font)
        self.designation.setStyleSheet("padding: 8px;")
        
        # Add fields to form
        form_layout.addRow(QLabel("Full Name:"), self.name)
        form_layout.addRow(QLabel("Email:"), self.email)
        form_layout.addRow(QLabel("Phone:"), self.phone)
        form_layout.addRow(QLabel("Designation:"), self.designation)
        
        # Update button
        self.updateBtn = QPushButton("Update")
        self.updateBtn.setFont(QFont("Arial", 12, QFont.Bold))
        self.updateBtn.setStyleSheet(
            "background-color: #f39c12; color: white; padding: 10px 30px; border-radius: 5px;"
        )
        self.updateBtn.clicked.connect(self.updateData)
        self.updateBtn.setEnabled(False)  # Disabled until employee is found
        
        # Add widgets to main layout
        main_layout.addWidget(search_group)
        main_layout.addWidget(form_group)
        main_layout.addWidget(self.updateBtn, alignment=Qt.AlignCenter)
        main_layout.addStretch(1)
        
    def searchEmployee(self):
        """Search for employee by ID and populate form fields"""
        emp_id = self.empid.text().strip()
        if not emp_id:
            QMessageBox.warning(self, "Error", "Please enter Employee ID to search.")
            return
            
        try:
            # Convert to integer for database lookup
            emp_id = int(emp_id)
        except ValueError:
            QMessageBox.warning(self, "Error", "Employee ID must be a number.")
            return

        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Search for employee
            query = "SELECT name, email, phone, Designation FROM emp WHERE id = %s"
            cur.execute(query, (emp_id,))
            employee = cur.fetchone()
            
            if employee:
                # Populate form with existing data
                self.name.setText(employee[0])
                self.email.setText(employee[1])
                self.phone.setText(str(employee[2]))
                
                # Set designation in combobox
                index = self.designation.findText(employee[3])
                if index >= 0:
                    self.designation.setCurrentIndex(index)
                else:
                    self.designation.setCurrentText("Other")
                    
                self.updateBtn.setEnabled(True)  # Enable update button
            else:
                QMessageBox.warning(self, "Not Found", f"No employee found with ID: {emp_id}")
                self.updateBtn.setEnabled(False)
            
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
            self.updateBtn.setEnabled(False)
        finally:
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()

    def updateData(self):
        """Update employee record in database"""
        # Validate inputs
        emp_id = self.empid.text().strip()
        if not emp_id:
            QMessageBox.warning(self, "Error", "Employee ID is required.")
            return
            
        try:
            # Convert to integer for database
            emp_id = int(emp_id)
        except ValueError:
            QMessageBox.warning(self, "Error", "Employee ID must be a number.")
            return
            
        if not all([self.name.text().strip(), 
                    self.email.text().strip(), 
                    self.phone.text().strip()]):
            QMessageBox.warning(self, "Error", "Please fill all required fields!")
            return

        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Update query
            query = """
                UPDATE emp 
                SET name = %s, email = %s, phone = %s, Designation = %s 
                WHERE id = %s
            """
            values = (
                self.name.text(),
                self.email.text(),
                self.phone.text(),
                self.designation.currentText(),
                emp_id
            )
            
            cur.execute(query, values)
            conn.commit()  # Save changes
            
            if cur.rowcount > 0:
                QMessageBox.information(
                    self, 
                    "Success", 
                    f"Employee ID {emp_id} updated successfully!"
                )
            else:
                QMessageBox.warning(
                    self, 
                    "Not Updated", 
                    f"No employee found with ID: {emp_id}. Record not updated."
                )
            
        except mysql.connector.Error as err:
            QMessageBox.critical(
                self, 
                "Database Error", 
                f"Error: {err}"
            )
        finally:
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()


class DeleteEmployeeWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        main_layout = QVBoxLayout()
        self.setLayout(main_layout)
        
        # Title
        title = QLabel("Delete Employee")
        title.setFont(QFont("Arial", 18, QFont.Bold))
        title.setStyleSheet("color: #2c3e50; margin: 10px 0 20px 0;")
        title.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(title)
        
        # Search section
        search_group = QGroupBox("Search Employee")
        search_group.setFont(QFont("Arial", 12))
        search_layout = QFormLayout()
        search_layout.setSpacing(15)
        search_layout.setContentsMargins(20, 20, 20, 20)
        search_group.setLayout(search_layout)
        
        # Search input
        self.empid = QLineEdit()
        self.empid.setPlaceholderText("Enter Employee ID")
        self.empid.setFont(QFont("Arial", 11))
        self.empid.setStyleSheet("padding: 8px;")
        
        # Add to form
        search_layout.addRow(QLabel("Employee ID:"), self.empid)
        
        # Delete button
        self.deleteBtn = QPushButton("Delete Employee")
        self.deleteBtn.setFont(QFont("Arial", 12, QFont.Bold))
        self.deleteBtn.setStyleSheet(
            "background-color: #e74c3c; color: white; padding: 10px 30px; border-radius: 5px;"
        )
        self.deleteBtn.clicked.connect(self.deleteEmployee)
        
        # Add widgets to main layout
        main_layout.addWidget(search_group)
        main_layout.addWidget(self.deleteBtn, alignment=Qt.AlignCenter)
        main_layout.addStretch(1)
        
    def deleteEmployee(self):
        emp_id = self.empid.text().strip()
        
        # Validate input
        if not emp_id:
            QMessageBox.warning(self, "Error", "Please enter Employee ID.")
            return
            
        try:
            # Convert to integer
            emp_id = int(emp_id)
        except ValueError:
            QMessageBox.warning(self, "Error", "Employee ID must be a number.")
            return

        # Confirm deletion
        reply = QMessageBox.question(
            self,
            "Confirm Deletion",
            f"Are you sure you want to delete employee with ID {emp_id}?\nThis action cannot be undone.",
            QMessageBox.Yes | QMessageBox.No,
            QMessageBox.No
        )
        
        if reply != QMessageBox.Yes:
            return

        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Delete query using parameterized statement
            query = "DELETE FROM emp WHERE id = %s"
            cur.execute(query, (emp_id,))
            
            # Check if any row was affected
            if cur.rowcount > 0:
                QMessageBox.information(
                    self, 
                    "Deleted", 
                    f"Employee with ID {emp_id} deleted successfully!"
                )
                # Clear input after successful deletion
                self.empid.clear()
            else:
                QMessageBox.warning(
                    self, 
                    "Not Found", 
                    f"No employee found with ID: {emp_id}. Nothing deleted."
                )
                
            conn.commit()  # Save changes
            
        except mysql.connector.Error as err:
            QMessageBox.critical(
                self, 
                "Database Error", 
                f"Error: {err}"
            )
        finally:
            # Clean up resources
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()


class ViewProfileWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        main_layout = QVBoxLayout()
        self.setLayout(main_layout)
        
        # Title
        title = QLabel("View Employee Profile")
        title.setFont(QFont("Arial", 18, QFont.Bold))
        title.setStyleSheet("color: #2c3e50; margin: 10px 0 20px 0;")
        title.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(title)
        
        # Search section
        search_group = QGroupBox("Search Employee")
        search_group.setFont(QFont("Arial", 12))
        search_layout = QFormLayout()
        search_layout.setSpacing(15)
        search_layout.setContentsMargins(20, 20, 20, 20)
        search_group.setLayout(search_layout)
        
        # Search input
        self.empid = QLineEdit()
        self.empid.setPlaceholderText("Enter Employee ID")
        self.empid.setFont(QFont("Arial", 11))
        self.empid.setStyleSheet("padding: 8px;")
        
        # View button
        self.viewBtn = QPushButton("View Profile")
        self.viewBtn.setFont(QFont("Arial", 11))
        self.viewBtn.setStyleSheet(
            "background-color: #3498db; color: white; padding: 8px 20px; border-radius: 5px;"
        )
        self.viewBtn.clicked.connect(self.displayDetails)
        
        # Add to form
        search_layout.addRow(QLabel("Employee ID:"), self.empid)
        search_layout.addRow(self.viewBtn)
        
        # Profile display section
        profile_group = QGroupBox("Employee Profile")
        profile_group.setFont(QFont("Arial", 12))
        profile_layout = QVBoxLayout()
        profile_layout.setContentsMargins(20, 20, 20, 20)
        profile_group.setLayout(profile_layout)
        
        self.profile = QTextEdit()
        self.profile.setFont(QFont("Arial", 12))
        self.profile.setReadOnly(True)
        self.profile.setStyleSheet("background-color: #f8f9fa; border: none; padding: 15px;")
        
        profile_layout.addWidget(self.profile)
        
        # Add widgets to main layout
        main_layout.addWidget(search_group)
        main_layout.addWidget(profile_group)
        
    def displayDetails(self):
        emp_id = self.empid.text().strip()
        if not emp_id:
            QMessageBox.warning(self, "Error", "Please enter Employee ID.")
            return
            
        try:
            # Convert to integer
            emp_id = int(emp_id)
        except ValueError:
            QMessageBox.warning(self, "Error", "Employee ID must be a number.")
            return

        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Fetch employee details by ID
            query = """
                SELECT id, name, email, phone, Designation 
                FROM emp 
                WHERE id = %s
            """
            cur.execute(query, (emp_id,))
            employee = cur.fetchone()
            
            if employee:
                # Format the result
                details = (
                    f"<h2 style='color: #2980b9;'>Employee Profile: #{employee[0]}</h2>"
                    "<hr>"
                    f"<p><b>Name:</b> {employee[1]}</p>"
                    f"<p><b>Email:</b> {employee[2]}</p>"
                    f"<p><b>Phone:</b> {employee[3]}</p>"
                    f"<p><b>Designation:</b> {employee[4]}</p>"
                )
                self.profile.setHtml(details)
            else:
                self.profile.setHtml("<p style='color: #e74c3c;'>No employee found with this ID.</p>")
                
        except mysql.connector.Error as err:
            self.profile.setHtml(f"<p style='color: #e74c3c;'>Database Error: {err}</p>")
        finally:
            # Clean up resources
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()


class EmployeeListWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        main_layout = QVBoxLayout()
        self.setLayout(main_layout)
        
        # Title
        title = QLabel("Employee Directory")
        title.setFont(QFont("Arial", 18, QFont.Bold))
        title.setStyleSheet("color: #2c3e50; margin: 10px 0 20px 0;")
        title.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(title)
        
        # Search bar
        search_layout = QHBoxLayout()
        
        self.search_input = QLineEdit()
        self.search_input.setPlaceholderText("Search by name, email, or designation...")
        self.search_input.setFont(QFont("Arial", 11))
        self.search_input.setStyleSheet("padding: 8px;")
        
        self.search_btn = QPushButton("Search")
        self.search_btn.setFont(QFont("Arial", 11))
        self.search_btn.setStyleSheet(
            "background-color: #3498db; color: white; padding: 8px 20px; border-radius: 5px;"
        )
        self.search_btn.clicked.connect(self.loadData)
        
        self.refresh_btn = QPushButton("Refresh")
        self.refresh_btn.setFont(QFont("Arial", 11))
        self.refresh_btn.setStyleSheet(
            "background-color: #2ecc71; color: white; padding: 8px 20px; border-radius: 5px;"
        )
        self.refresh_btn.clicked.connect(self.loadData)
        
        search_layout.addWidget(self.search_input)
        search_layout.addWidget(self.search_btn)
        search_layout.addWidget(self.refresh_btn)
        
        # Employee table
        self.table = QTableWidget()
        self.table.setColumnCount(6)
        self.table.setHorizontalHeaderLabels([
            "ID", "Name", "Email", "Phone", "Designation", "Actions"
        ])
        
        # Style table headers
        header = self.table.horizontalHeader()
        header.setStyleSheet("""
            QHeaderView::section {
                background-color: #3498db;
                color: white;
                padding: 6px;
                font-weight: bold;
                border: none;
            }
        """)
        
        # Adjust column widths
        self.table.setColumnWidth(0, 80)   # ID
        self.table.setColumnWidth(1, 200)  # Name
        self.table.setColumnWidth(2, 250)  # Email
        self.table.setColumnWidth(3, 150)  # Phone
        self.table.setColumnWidth(4, 180)  # Designation
        self.table.setColumnWidth(5, 300)  # Actions
        
        # Enable sorting and row selection
        self.table.setSortingEnabled(True)
        self.table.setSelectionBehavior(QAbstractItemView.SelectRows)
        
        # Style the table
        self.table.setStyleSheet("""
            QTableWidget {
                background-color: white;
                border: 1px solid #ddd;
                border-radius: 5px;
                gridline-color: #eee;
            }
            QTableWidget::item {
                padding: 8px;
            }
            QTableWidget::item:selected {
                background-color: #d6eaf8;
                color: black;
            }
        """)
        
        # Add widgets to main layout
        main_layout.addLayout(search_layout)
        main_layout.addWidget(self.table)
        
        # Load data initially
        self.loadData()
        
    def loadData(self):
        try:
            # Connect to database
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='employee'
            )
            cur = conn.cursor()
            
            # Build query based on search
            search_term = self.search_input.text().strip()
            if search_term:
                query = """
                    SELECT id, name, email, phone, Designation 
                    FROM emp 
                    WHERE name LIKE %s OR email LIKE %s OR Designation LIKE %s
                """
                search_term = f"%{search_term}%"
                cur.execute(query, (search_term, search_term, search_term))
            else:
                query = "SELECT id, name, email, phone, Designation FROM emp"
                cur.execute(query)
                
            employees = cur.fetchall()
            
            if not employees:
                self.table.setRowCount(0)
                return
                
            self.table.setRowCount(len(employees))
            
            for row, employee in enumerate(employees):
                # Add data to columns 0-4
                for col in range(5):
                    item = QTableWidgetItem(str(employee[col]))
                    item.setFlags(item.flags() ^ Qt.ItemIsEditable)  # Make non-editable
                    self.table.setItem(row, col, item)
                
                # Create action buttons
                btn_frame = QWidget()
                btn_layout = QHBoxLayout()
                btn_layout.setContentsMargins(5, 5, 5, 5)
                btn_layout.setSpacing(5)
                
                # View button
                viewBtn = QPushButton("View")
                viewBtn.setFixedSize(70, 30)
                viewBtn.setStyleSheet("background-color: #3498db; color: white; border-radius: 4px;")
                viewBtn.clicked.connect(lambda _, eid=employee[0]: self.viewProfile(eid))
                btn_layout.addWidget(viewBtn)
                
                # Update button
                updateBtn = QPushButton("Update")
                updateBtn.setFixedSize(70, 30)
                updateBtn.setStyleSheet("background-color: #f39c12; color: white; border-radius: 4px;")
                updateBtn.clicked.connect(lambda _, eid=employee[0]: self.updateEmp(eid))
                btn_layout.addWidget(updateBtn)
                
                # Delete button
                deleteBtn = QPushButton("Delete")
                deleteBtn.setFixedSize(70, 30)
                deleteBtn.setStyleSheet("background-color: #e74c3c; color: white; border-radius: 4px;")
                deleteBtn.clicked.connect(lambda _, eid=employee[0]: self.deleteEmp(eid))
                btn_layout.addWidget(deleteBtn)
                
                btn_frame.setLayout(btn_layout)
                self.table.setCellWidget(row, 5, btn_frame)
                
            # Resize rows to fit content
            self.table.resizeRowsToContents()
                
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
        finally:
            # Clean up resources
            if 'conn' in locals() and conn.is_connected():
                cur.close()
                conn.close()

    def viewProfile(self, eid):
        # Create and show profile window
        self.profile_window = ViewProfileWidget()
        self.profile_window.empid.setText(str(eid))
        self.profile_window.displayDetails()
        self.profile_window.show()

    def updateEmp(self, eid):
        # Create and show update window
        self.update_window = UpdateEmployeeWidget()
        self.update_window.empid.setText(str(eid))
        self.update_window.searchEmployee()
        self.update_window.show()

    def deleteEmp(self, eid):
        # Create and show delete window
        self.delete_window = DeleteEmployeeWidget()
        self.delete_window.empid.setText(str(eid))
        self.delete_window.deleteEmployee()
        # Refresh table after deletion
        self.loadData()


class AboutDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("About")
        self.setFixedSize(500, 400)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        # Header
        header = QLabel("ABC Employee Management System")
        header.setFont(QFont("Arial", 18, QFont.Bold))
        header.setAlignment(Qt.AlignCenter)
        header.setStyleSheet("color: #2c3e50; margin-bottom: 20px;")
        layout.addWidget(header)
        
        # Version
        version = QLabel("Version 2.0")
        version.setFont(QFont("Arial", 12))
        version.setAlignment(Qt.AlignCenter)
        layout.addWidget(version)
        
        # Description
        desc = QLabel(
            "A comprehensive solution for managing your organization's workforce.\n"
            "This application provides tools for adding, updating, and viewing\n"
            "employee records with an intuitive interface."
        )
        desc.setFont(QFont("Arial", 10))
        desc.setAlignment(Qt.AlignCenter)
        desc.setStyleSheet("margin: 20px 0;")
        layout.addWidget(desc)
        
        # Features
        features = QGroupBox("Features")
        features_layout = QVBoxLayout()
        
        feature_items = [
            "✓ Employee record management",
            "✓ Search and filter capabilities",
            "✓ Responsive and modern UI",
            "✓ Secure database operations",
            "✓ Easy navigation between functions"
        ]
        
        for item in feature_items:
            label = QLabel(item)
            label.setFont(QFont("Arial", 10))
            features_layout.addWidget(label)
            
        features.setLayout(features_layout)
        layout.addWidget(features)
        
        # Buttons
        buttons = QDialogButtonBox(QDialogButtonBox.Ok)
        buttons.accepted.connect(self.accept)
        layout.addWidget(buttons)


class DeveloperDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("About Developer")
        self.setFixedSize(500, 400)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        # Header
        header = QLabel("About the Developer")
        header.setFont(QFont("Arial", 18, QFont.Bold))
        header.setAlignment(Qt.AlignCenter)
        header.setStyleSheet("color: #2c3e50; margin-bottom: 20px;")
        layout.addWidget(header)
        
        # Developer info
        dev_info = QLabel(
            "This application was developed by:\n\n"
            "<b>John Smith</b>\n"
            "M.Sc. Computer Science\n"
            "University of Technology\n\n"
            "Specializing in database systems and UI design."
        )
        dev_info.setFont(QFont("Arial", 12))
        dev_info.setAlignment(Qt.AlignCenter)
        layout.addWidget(dev_info)
        
        # Contact
        contact = QLabel(
            "<b>Contact:</b> john.smith@example.com\n"
            "<b>Website:</b> www.johnsmith.dev"
        )
        contact.setFont(QFont("Arial", 10))
        contact.setAlignment(Qt.AlignCenter)
        contact.setStyleSheet("margin: 20px 0;")
        layout.addWidget(contact)
        
        # Skills
        skills = QGroupBox("Technical Skills")
        skills_layout = QVBoxLayout()
        
        skill_items = [
            "✓ Python programming",
            "✓ PyQt5 framework",
            "✓ MySQL database design",
            "✓ UI/UX design principles",
            "✓ Software architecture"
        ]
        
        for item in skill_items:
            label = QLabel(item)
            label.setFont(QFont("Arial", 10))
            skills_layout.addWidget(label)
            
        skills.setLayout(skills_layout)
        layout.addWidget(skills)
        
        # Buttons
        buttons = QDialogButtonBox(QDialogButtonBox.Ok)
        buttons.accepted.connect(self.accept)
        layout.addWidget(buttons)


class DatabaseDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Database Information")
        self.setFixedSize(500, 350)
        
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        # Header
        header = QLabel("Database Connection")
        header.setFont(QFont("Arial", 18, QFont.Bold))
        header.setAlignment(Qt.AlignCenter)
        header.setStyleSheet("color: #2c3e50; margin-bottom: 20px;")
        layout.addWidget(header)
        
        # Info
        info = QLabel(
            "This application uses a MySQL database to store employee records.\n\n"
            "The database is securely connected using the mysql.connector module."
        )
        info.setFont(QFont("Arial", 11))
        info.setAlignment(Qt.AlignCenter)
        info.setStyleSheet("margin-bottom: 20px;")
        layout.addWidget(info)
        
        # Connection details
        details = QGroupBox("Connection Details")
        details_layout = QFormLayout()
        
        detail_items = [
            ("Host:", "localhost"),
            ("Database:", "employee"),
            ("User:", "root"),
            ("Password:", "Not displayed for security"),
            ("Driver:", "mysql.connector")
        ]
        
        for label, value in detail_items:
            lbl = QLabel(label)
            lbl.setFont(QFont("Arial", 10, QFont.Bold))
            
            val = QLabel(value)
            val.setFont(QFont("Arial", 10))
            
            details_layout.addRow(lbl, val)
            
        details.setLayout(details_layout)
        layout.addWidget(details)
        
        # Buttons
        buttons = QDialogButtonBox(QDialogButtonBox.Ok)
        buttons.accepted.connect(self.accept)
        layout.addWidget(buttons)


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("ABC COMPANY - EMPLOYEE MANAGEMENT SYSTEM")
        self.setGeometry(100, 100, 1200, 700)
        
        # Apply application style
        self.setStyle()
        
        # Create stacked widget for views
        self.stacked_widget = QStackedWidget()
        self.setCentralWidget(self.stacked_widget)
        
        # Create all views
        self.welcome_page = WelcomePage()
        self.add_emp_widget = AddEmployeeWidget()
        self.update_emp_widget = UpdateEmployeeWidget()
        self.delete_emp_widget = DeleteEmployeeWidget()
        self.view_profile_widget = ViewProfileWidget()
        self.employee_list_widget = EmployeeListWidget()
        
        # Add views to stacked widget
        self.stacked_widget.addWidget(self.welcome_page)
        self.stacked_widget.addWidget(self.add_emp_widget)
        self.stacked_widget.addWidget(self.update_emp_widget)
        self.stacked_widget.addWidget(self.delete_emp_widget)
        self.stacked_widget.addWidget(self.view_profile_widget)
        self.stacked_widget.addWidget(self.employee_list_widget)
        
        # Create menu bar
        self.createMenuBar()
        
        # Create toolbar
        self.createToolBar()
        
        # Create status bar
        self.statusBar().showMessage("Ready")
        
        # Show the welcome page initially
        self.showWelcomePage()
        
    def setStyle(self):
        # Set application-wide style
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f5f7fa;
            }
            QGroupBox {
                background-color: white;
                border: 1px solid #d1d5db;
                border-radius: 8px;
                margin-top: 20px;
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 10px;
                padding: 0 5px;
                color: #3498db;
            }
            QPushButton:hover {
                opacity: 0.9;
            }
            QLineEdit, QComboBox {
                border: 1px solid #d1d5db;
                border-radius: 4px;
                padding: 5px;
            }
            QLineEdit:focus, QComboBox:focus {
                border: 1px solid #3498db;
            }
        """)
        
        # Set application palette
        palette = QPalette()
        palette.setColor(QPalette.Window, QColor("#f5f7fa"))
        palette.setColor(QPalette.WindowText, Qt.black)
        palette.setColor(QPalette.Base, QColor("#ffffff"))
        palette.setColor(QPalette.AlternateBase, QColor("#f0f4f8"))
        palette.setColor(QPalette.ToolTipBase, QColor("#2c3e50"))
        palette.setColor(QPalette.ToolTipText, Qt.white)
        palette.setColor(QPalette.Text, Qt.black)
        palette.setColor(QPalette.Button, QColor("#3498db"))
        palette.setColor(QPalette.ButtonText, Qt.white)
        palette.setColor(QPalette.BrightText, Qt.red)
        palette.setColor(QPalette.Highlight, QColor("#3498db"))
        palette.setColor(QPalette.HighlightedText, Qt.white)
        self.setPalette(palette)
    
    def createMenuBar(self):
        menubar = self.menuBar()
        
        # Employee menu
        emp_menu = menubar.addMenu("Employee")
        
        # Add Employee
        add_action = QAction("Add New Employee", self)
        add_action.triggered.connect(self.showAddEmployee)
        emp_menu.addAction(add_action)
        
        # Update Employee
        update_action = QAction("Update Employee", self)
        update_action.triggered.connect(self.showUpdateEmployee)
        emp_menu.addAction(update_action)
        
        # Delete Employee
        delete_action = QAction("Delete Employee", self)
        delete_action.triggered.connect(self.showDeleteEmployee)
        emp_menu.addAction(delete_action)
        
        # Separator
        emp_menu.addSeparator()
        
        # View Employee submenu
        view_submenu = QMenu("View Employee", self)
        
        # All Employee List
        all_emp_action = QAction("All Employee List", self)
        all_emp_action.triggered.connect(self.showEmployeeList)
        view_submenu.addAction(all_emp_action)
        
        # View Profile
        profile_action = QAction("View Profile", self)
        profile_action.triggered.connect(self.showViewProfile)
        view_submenu.addAction(profile_action)
        
        emp_menu.addMenu(view_submenu)
        
        # Help menu
        help_menu = menubar.addMenu("Help")
        
        # About App
        about_app_action = QAction("About Application", self)
        about_app_action.triggered.connect(self.showAboutApp)
        help_menu.addAction(about_app_action)
        
        # About Developer
        about_dev_action = QAction("About Developer", self)
        about_dev_action.triggered.connect(self.showAboutDeveloper)
        help_menu.addAction(about_dev_action)
        
        # Database Info
        db_action = QAction("Database Connection", self)
        db_action.triggered.connect(self.showDatabaseInfo)
        help_menu.addAction(db_action)
        
        # Separator
        help_menu.addSeparator()
        
        # Exit
        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        help_menu.addAction(exit_action)
    
    def createToolBar(self):
        toolbar = QToolBar("Main Toolbar")
        toolbar.setIconSize(QSize(32, 32))
        self.addToolBar(toolbar)
        
        # Home button
        home_action = QAction(QIcon("icons/home.png"), "Home", self)
        home_action.triggered.connect(self.showWelcomePage)
        toolbar.addAction(home_action)
        
        # Add separator
        toolbar.addSeparator()
        
        # Add Employee
        add_action = QAction(QIcon("icons/add_user.png"), "Add Employee", self)
        add_action.triggered.connect(self.showAddEmployee)
        toolbar.addAction(add_action)
        
        # Update Employee
        update_action = QAction(QIcon("icons/edit.png"), "Update Employee", self)
        update_action.triggered.connect(self.showUpdateEmployee)
        toolbar.addAction(update_action)
        
        # Delete Employee
        delete_action = QAction(QIcon("icons/delete.png"), "Delete Employee", self)
        delete_action.triggered.connect(self.showDeleteEmployee)
        toolbar.addAction(delete_action)
        
        # View Employee List
        list_action = QAction(QIcon("icons/list.png"), "Employee List", self)
        list_action.triggered.connect(self.showEmployeeList)
        toolbar.addAction(list_action)
        
        # Add separator
        toolbar.addSeparator()
        
        # Exit
        exit_action = QAction(QIcon("icons/exit.png"), "Exit", self)
        exit_action.triggered.connect(self.close)
        toolbar.addAction(exit_action)
    
    def showWelcomePage(self):
        self.stacked_widget.setCurrentWidget(self.welcome_page)
        self.statusBar().showMessage("Welcome to ABC Employee Management System")
    
    def showAddEmployee(self):
        self.stacked_widget.setCurrentWidget(self.add_emp_widget)
        self.statusBar().showMessage("Add new employee to the system")
    
    def showUpdateEmployee(self):
        self.stacked_widget.setCurrentWidget(self.update_emp_widget)
        self.statusBar().showMessage("Update existing employee information")
    
    def showDeleteEmployee(self):
        self.stacked_widget.setCurrentWidget(self.delete_emp_widget)
        self.statusBar().showMessage("Delete an employee from the system")
    
    def showViewProfile(self):
        self.stacked_widget.setCurrentWidget(self.view_profile_widget)
        self.statusBar().showMessage("View employee profile")
    
    def showEmployeeList(self):
        self.employee_list_widget.loadData()
        self.stacked_widget.setCurrentWidget(self.employee_list_widget)
        self.statusBar().showMessage("Viewing all employees")
    
    def showAboutApp(self):
        dialog = AboutDialog(self)
        dialog.exec_()
    
    def showAboutDeveloper(self):
        dialog = DeveloperDialog(self)
        dialog.exec_()
    
    def showDatabaseInfo(self):
        dialog = DatabaseDialog(self)
        dialog.exec_()


if __name__ == "__main__":
    # Create database if it doesn't exist
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            password=''
        )
        cur = conn.cursor()
        
        # Create database
        cur.execute("CREATE DATABASE IF NOT EXISTS employee")
        cur.execute("USE employee")
        
        # Create table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS emp (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                email VARCHAR(255) NOT NULL,
                phone VARCHAR(20) NOT NULL,
                Designation VARCHAR(100) NOT NULL
            )
        """)
        
        # Insert sample data if table is empty
        cur.execute("SELECT COUNT(*) FROM emp")
        count = cur.fetchone()[0]
        
        if count == 0:
            sample_data = [
                ("John Doe", "john.doe@example.com", "555-1234", "Developer"),
                ("Jane Smith", "jane.smith@example.com", "555-5678", "HR"),
                ("Robert Johnson", "robert.j@example.com", "555-9012", "Team Leader"),
                ("Emily Davis", "emily.d@example.com", "555-3456", "Designer"),
                ("Michael Wilson", "michael.w@example.com", "555-7890", "Tester")
            ]
            
            query = """
                INSERT INTO emp 
                (name, email, phone, Designation) 
                VALUES (%s, %s, %s, %s)
            """
            
            cur.executemany(query, sample_data)
            conn.commit()
            
        conn.close()
    except mysql.connector.Error as err:
        print(f"Database setup error: {err}")
    
    # Run the application
    app = QApplication(sys.argv)
    app.setStyle("Fusion")  # Use Fusion style for a modern look
    
    # Set application font
    font = QFont("Arial", 10)
    app.setFont(font)
    
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\gurup\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
